# Object Detection
In this notebook we will have a look at the fridge dataset using different dashboards.

In [ ]:
# Imports
import pathlib

import panel as pn

from icevision.all import *
import icedata
from icevision_dashboards.dashboards import *
from icevision_dashboards.data import *

In [ ]:
pn.extension()

# Setup data
First we load the data in the typical icevision fashion

In [ ]:
# Load the Fridge dataset
path = icedata.fridge.load_data()

# get the class map
class_map = icedata.fridge.class_map()

# parse the data
parser = icedata.fridge.parser(data_dir=path)
# we just want to have a look at the data so we don't split the data
records = parser.parse(RandomSplitter([1]))[0]

## Prepare the data for the dashboard
To use the records with various dashboards we need to first create dataset that contains all the required data. For this the `BboxRecordDataset` can be used (to learn more about `datasets` and `dashboards` take a look at the notebooks: N1, N2), it takes a list of records and calulates different statistics which can be used by the dashboards.

In [ ]:
# create a dataset that can be consumed by a dashboard
# use the class map for nicer representation
fridge_ds = BboxRecordDataset(records, class_map)

The overview has four tabs that display different information about the dataset.

1. Dataset stats: General statistics about the dataset (images, and classes)
2. Annotations: Plots to investigate the distributions of diffrent aspects
3. Gallery: A sortable gallery of the images in the dataset
4. Dataset: A table containing the reach annotation as a row and corrospoinding stats as the columns

In [ ]:
# create dashboard that gives us an overview of the dataset
fridge_overview = ObjectDetectionDatasetOverview(fridge_ds, 700, 1200)
# every dashboard has a show function to display the dashboard
# if only a list with text is showing you might have forgotten to run pn.extension() in a cell
fridge_overview.show()

# Comparing datasets
We split the records into training and validation records and compare the two datasets. The comparison dashboard is very similar to the Overview dashboard but it allows for linking plots the make them comparable. 

In [ ]:
# lets split the data and compare the two
train_records, valid_records = parser.parse()

In [ ]:
# create two datasets for the comparison dashboard
dash_ds_train = BboxRecordDataset(train_records, class_map, name="fridge_train")
dash_ds_valid = BboxRecordDataset(valid_records, class_map, name="fridge_valid")

In [ ]:
# save the two datasets for later use
# dash_ds_train.save("test_data/")
# dash_ds_valid.save("test_data/")

In [ ]:
# create the comparison dashboard and display it
comparison_dashboard = ObjectDetectionDatasetComparison([dash_ds_train, dash_ds_valid], 500, 1500)
comparison_dashboard.show()

# Train model
The validation and trainings dataset look good, so we can start training.

In [ ]:
# Define transforms
train_tfms = tfms.A.Adapter(
    [*tfms.A.aug_tfms(size=384, presize=512), tfms.A.Normalize()]
)
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(384), tfms.A.Normalize()])

In [ ]:
# Create datasets for training
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

## Model Setup

Below the different models and libaries are provided so you can test the result dashboard for different models/libaries. Just uncomment the model you want to test and comment out all other models. Some models might require additional installs (see the icevison installation guide).

### Torchvision - Faster RCNN

In [ ]:
# model_type = models.torchvision.faster_rcnn
# backbone = model_type.backbones.resnet18_fpn(pretrained=True)
# extra_params = {}

### Torchvision - Retinanet

In [ ]:
# model_type = models.torchvision.retinanet
# backbone = model_type.backbones.resnet18_fpn(pretrained=True)
# extra_params = {}

### MMLab - FCOS

In [ ]:
# model_type = models.mmdet.fcos
# backbone = model_type.backbones.resnet50_caffe_fpn_gn_head_dcn_1x_center_normbbox_centeronreg_giou(pretrained=True)
# extra_params = {}

### MMLab - Faster RCNN

In [ ]:
# model_type = models.mmdet.faster_rcnn
# backbone = model_type.backbones.resnet50_fpn_1x(pretrained=True)
# extra_params = {}

### MMLab - VFNet

In [ ]:
model_type = models.mmdet.vfnet
backbone = model_type.backbones.resnet50_fpn_1x(pretrained=True)
extra_params = {}

### MMLab - Retinanet

In [ ]:
# model_type = models.mmdet.retinanet
# backbone = model_type.backbones.resnet50_fpn_1x(pretrained=True)
# extra_params = {}

### MMLab - Sparse RCNN (broken)

In [ ]:
# model_type = models.mmdet.sparse_rcnn
# backbone = model_type.backbones.resnet50_fpn_1x(pretrained=True)
# extra_params = {}

### TIMM - EfficientDet (broken)

In [ ]:
# model_type = models.efficientdet
# backbone = model_type.backbones.tf_d6(pretrained=True)
# extra_params = {"img_size": 384}

### Build model

In [ ]:
# Create model and train it using fastai2
model = model_type.model(backbone=backbone, num_classes=len(class_map), **extra_params)
metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

# Create dataloaders from the datasets
train_dl = model_type.train_dl(train_ds, batch_size=1, num_workers=4, shuffle=True)
valid_dl = model_type.valid_dl(valid_ds, batch_size=1, num_workers=4, shuffle=False)

learn = model_type.fastai.learner(
    dls=[train_dl, valid_dl], model=model, metrics=metrics
)

learn.fine_tune(5, freeze_epochs=3, base_lr=1e-4)

## Analyse the result

In [ ]:
# we can now get the losses and predictions of the validation dataset
samples, preds, losses_stats = model_type.interp.plot_top_losses(model=model, dataset=valid_ds, sort_by="loss_total", n_samples=1, batch_size=2)

In [ ]:
# first create a dataset that can be consumed by the analysis dashboard
valid_result_ds = ObjectDetectionResultsDataset.init_from_preds_and_samples(preds, samples, class_map=class_map)

## Saving and loading a result dataset
We can save the result data for further evaluation at a later point.

In [ ]:
# save the results
valid_result_ds.save("datasets/object_detection_result_ds.dat")

In [ ]:
# load the results
valid_result_ds = ObjectDetectionResultsDataset.load("datasets/object_detection_result_ds.dat")

In [ ]:
# show the results in a dashboard
result_overview_dash = ObjectDetectionResultOverview(valid_result_ds, width=1500)
result_overview_dash.show()

Because the recalculation of the layout can take some time the two main tabs can also be shown independenly which given a better performance for the interative plots. Make sure that you clear the output from the cell that shows the full dashboard, because the dashboards are the same and an update to one also transfers to the other.

In [ ]:
# only show the loss tab
# result_overview_dash.show_loss_tab()

In [ ]:
# only show the ap tab
# result_overview_dash.show_ap_tab()